# Team 10 Homework 5

## CS02620 Section 1

- Leonardo Florero-Flores
- Jason Nichols
- Fru Nkimberg
- Harman Ransi


# Deliverable 1: Sales summary table


## Combined and Per Store: 
- Total sales ($)

## Per Store
- Total sales ($)
- Top 25 SKU, Product Name, Number Sold, Rank


In [74]:
#
# Let's start by loading up transactions for each store.  We've got different file formats so let's normalize each 
# into a list of lists: date, customer #, sku, sale price
#
team_sales = {}

#
# Team 10
#
# Team 10 used a sqlite file, one that includes inventory data
#
import sqlite3
key = 'team10'
with sqlite3.connect("data/team10.db") as con:
    cur = con.execute("SELECT Date, CustomerNum, SKU, SalePrice FROM transactions")
    team_sales[key] = [(row[0], str(row[1]), row[2], row[3]) for row in cur.fetchall()]
print(team_sales[key][0])
    
#
# Team 11
#
# Team 11 used a pipe-separated txt file without a header row
#
key = 'team11'
with open('data/team11.txt') as csvfile:
    team_sales[key] = [(row[0], row[1], row[2], float(row[3])) for row in csv.reader(csvfile, delimiter='|')]
print(team_sales[key][0])


#
# Team 12
#
# Team 12 used a CSV with a header row
#
import csv
key = 'team12'
with open('data/team12.csv') as csvfile:
    team_sales[key] = [(row[0], row[1], row[2], row[3]) for row in csv.reader(csvfile)]
team_sales[key].pop(0) # Remove the header (Python's csv.reader() doesn't seem to offer this feature)
team_sales[key] = [(row[0], row[1], row[2], float(row[3])) for row in team_sales[key]]  # Convert the last col to float
print(team_sales[key][0])


#
# Team 13
#
# Team 13 used a CSV with a header row
#
import csv
key = 'team13'
with open('data/team13.csv') as csvfile:
    team_sales[key] = [(row[0], row[1], row[2], row[3]) for row in csv.reader(csvfile)]
team_sales[key].pop(0) # Remove the header (Python's csv.reader() doesn't seem to offer this feature)
team_sales[key] = [(row[0], row[1], row[2], float(row[3])) for row in team_sales[key]]  # Convert the last col to float
print(team_sales[key][0])



('20230101', '1', '42358001', 2.17)
('20230101', '968', '42486001', 1.22)
('20230101', '1', '43686001', 2.19)
('01-01-2020', '0', '42355001', 3.87)


# Data Normalization

Team 13 is using a different date format than Teams 10, 11, and 12.  Let's correct that now.

In [75]:
def team13_date(date_str):
  return f"{date_str[-4:]}{date_str[:2]}{date_str[3:5]}"    

team_sales[key] = [(team13_date(row[0]), row[1], row[2], row[3]) for row in team_sales["team13"]]
print(team_sales[key][0])


('20200101', '0', '42355001', 3.87)


# Sales Summary

In [85]:
#
# Let's start with combined sales
#
def format_currency(val):
    return '${:,.2f}'.format(val)

sales = {}
for team, tx in team_sales.items():
    sales[team] = round(sum([t[3] for t in tx]), 2)  # Sum all sales (index 3) and round to two decimals
sales_total = round(sum(sales.values()),2)


import pandas as pd
from IPython.display import display, HTML
sales_list = [format_currency(sales_total)]
sales_stores = ["Total"]
for team, ts in sales.items():
    sales_list.append(format_currency(ts))
    sales_stores.append(team)
frame = pd.DataFrame([sales_list], columns=sales_stores )
display(HTML(frame.to_html()))

,Total,team10,team11,team12,team13
0,"$408,675,678.70","$49,082,383.78","$316,419,651.93","$39,766,472.90","$3,407,170.09"


# Best Selling Items

In [98]:

# Build the sales by sku dicts
#
total_sales_by_sku = {}  # sku->sales
team_sales_by_sku = {} # team->sku->sales

for team, transactions in team_sales.items():
    for tx in transactions:
        sku = tx[2]
        sku_sales = tx[3]

        # Total
        sales_by_sku = total_sales_by_sku.get(sku, 0.0)
        sales_by_sku += sku_sales
        total_sales_by_sku[sku] = sales_by_sku
    
        # Team
        sku_dict = team_sales_by_sku.get(team, dict())
        sales_by_sku = sku_dict.get(sku, 0.0)
        sales_by_sku += sku_sales
        sku_dict[sku] = sales_by_sku
        team_sales_by_sku[team] = sku_dict
 

# print(total_sales_by_sku)

#
# Sort total sales by sku and grab the top 25
#
def value_getter(item):
  return item[1]

skus_by_total_sales = sorted(total_sales_by_sku.items(), key=value_getter)
top_25_products = skus_by_total_sales[::-1][0:25]  # ordered list of sku:sales tuples (descending)


In [146]:
#
# Product Data
#
products = pd.read_csv("products.txt", encoding='unicode_escape', delimiter='|')

def by_sku(sku):
  res = products[products['SKU'] == sku]
  if (len(res) > 0):
        return res.iloc[0]
  return res

In [148]:
# Let's extract the Top 25 into a table

frame = pd.DataFrame(columns=['Product', 'SKU', 'Total','Team 10','Team 11','Team 12', 'Team 13'], index=list(range(0, 25)))
for i in range(0, 25):
    sku, total_sales = top_25_products[i]
    frame.loc[i] = [by_sku(int(sku))['Product Name'], sku, 
                    format_currency(total_sales), 
                    format_currency(team_sales_by_sku['team10'].get(sku, 0.0)), 
                    format_currency(team_sales_by_sku['team11'].get(sku, 0.0)), 
                    format_currency(team_sales_by_sku['team12'].get(sku, 0.0)), 
                    format_currency(team_sales_by_sku['team13'].get(sku, 0.0)), ]
display(HTML(frame.to_html()))

,Product,SKU,Total,Team 10,Team 11,Team 12,Team 13
0,Cruisers Size 5,42617001,"$1,451,138.36","$28,488.74","$1,366,715.10","$54,202.57","$1,731.95"
1,Baby Dry Size 5,42607001,"$1,344,058.33","$27,616.68","$1,267,770.87","$47,354.74","$1,316.04"
2,Cruisers Size 5,42615001,"$1,281,184.59","$27,035.84","$1,204,526.00","$47,355.55","$2,267.20"
3,Cruisers Size 5,42616001,"$1,280,302.01","$27,004.80","$1,206,337.10","$45,514.77","$1,445.34"
4,Coffee Talking Blue Medium,42781001,"$1,257,946.64","$183,921.14","$925,680.36","$135,099.42","$13,245.72"
5,Hamburgers Pure Beef,42510001,"$1,225,167.74","$178,977.15","$901,364.73","$131,896.02","$12,929.84"
6,Little Movers - Disney Size 6,43016001,"$1,097,759.60","$22,897.04","$1,032,639.78","$40,919.22","$1,303.56"
7,Little Snugglers Size Newborn,43024001,"$1,097,121.43","$22,262.48","$1,033,120.13","$40,362.84","$1,375.98"
8,Little Snugglers Size Newborn,43023001,"$1,091,857.63","$22,050.96","$1,029,604.79","$38,946.60","$1,255.28"
9,Snug & Dry - Disney Size 3,43031001,"$1,089,697.50","$23,002.80","$1,026,544.55","$38,870.73","$1,279.42"
